In [1]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
## 허깅페이스를 사용한 임베딩
from langchain_community.embeddings import HuggingFaceEmbeddings

d:\Shin\NLP\RAG\RAG_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path=r"D:\Shin\NLP\Corpus\namuwikitext\texts\namuwikitext_20200302-test.txt", encoding="utf-8")

docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print(f"문장의 수: {len(docs[0].page_content)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

문장의 수: 10849787

[메타데이터]

{'source': 'D:\\Shin\\NLP\\Corpus\\namuwikitext\\texts\\namuwikitext_20200302-test.txt'}

========= [앞부분] 미리보기 =========

 = 우오 켄 =

[목차]

 = 개요 =

魚 建
일본의 남자 성우.

 = 약력 =

1965년 7월 13일 일본 사이타마 현 출생

 = 출연 =

명탐정 코난 - 오구라 카츠마사 ( 2회 이상 조연 )
2013.11 - 아카자 나루야 ( 716~717 )
2018.10 -
틴 타이탄,틴 타이탄 GO! - 사이보그
포켓몬스터 썬&문 ( 2019 ) - 소푸우
후르츠 바스켓 - 혼다 카츠야의 아버지
이세계 치트 마술사 - 제라드
덤벨 몇 킬로까지 들 수 있어? - TV 장기자랑대회 프로듀서
ULTRAMAN - 이데 미츠히로

 = 박세웅(1996) =

 = = 개요 = =

삼성 라이온즈 소속의 좌완 투수.

 = = 선수 경력 = =

 = = = 아마추어 시절 = = =

한 해 선배인 황영국과 동기인 주권과 함께 2013년 청주고의 돌풍을 주도하면서 맹활약했다. 비록 창단 40년만에 처음으로 전국 대회 결승에 오른 청주고의 돌풍은 협회장기와 전국체전 준우승으로 끝나며 아


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
len(splits)

16489

In [5]:
model_name = "BAAI/bge-m3"     # 2024.02.21 embedding model 변경(기존: jhgan/ko-sroberta-multitask)
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    show_progress=True
)

# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=splits, embedding=hf)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

Batches: 100%|██████████| 516/516 [25:33<00:00,  2.97s/it] 


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [6]:

# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=splits, embedding=hf)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

Batches: 100%|██████████| 516/516 [25:25<00:00,  2.96s/it] 


In [23]:
vectorstore.save_local("namuwiki-test_db")

In [19]:
## 프롬프트 수정이 필요

## 키워드를 추출하는 프롬프트

prompt = hub.pull("godk/korean-rag")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an expert AI on a question and answer task. \nUse the "Following Context" when answering the question. If you don\'t know the answer, reply to the "Following Text" in the header and answer to the best of your knowledge, or if you do know the answer, answer without the "Following Text". If a question is asked in Korean, translate it to English and always answer in Korean.\nFollowing Text: "주어진 정보에서 답변을 찾지는 못했지만, 제가 아는 선에서 답을 말씀드려볼게요! **틀릴 수도 있으니 교차검증은 필수입니다!**"\n    \nFollowing Context: {context}\nQuestion: {question}\nHelpful Answer:'))])

In [20]:
print(prompt.messages[0].prompt.template)

You are an expert AI on a question and answer task. 
Use the "Following Context" when answering the question. If you don't know the answer, reply to the "Following Text" in the header and answer to the best of your knowledge, or if you do know the answer, answer without the "Following Text". If a question is asked in Korean, translate it to English and always answer in Korean.
Following Text: "주어진 정보에서 답변을 찾지는 못했지만, 제가 아는 선에서 답을 말씀드려볼게요! **틀릴 수도 있으니 교차검증은 필수입니다!**"
    
Following Context: {context}
Question: {question}
Helpful Answer:


In [21]:
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.0, convert_system_message_to_human=True)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
rag_chain.invoke(
    "부영그룹에 대해 설명해주세요."
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


IndexError: pop from empty list